In [1]:
import json 
import os 
import cv2
from pprint import pprint
import requests
import numpy as np
import random
import math
import albumentations as A
from tqdm import tqdm
import uuid
import requests
from matplotlib import pyplot as plt
%matplotlib inline

/home/hbdesk/labelstudio_convert/venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
LABEL_STUDIO_KEY="eaffb8f4719efb17f9b227ed56fee991c4d06a0a"
LABEL_STUDIO_URL="http://192.168.231.52:8080/"
OCR_SERVER="http://localhost:3000/en"


def string_from_img(image_path: str):
    with open(image_path, "rb") as image_file:
        files = { "file": (image_path, image_file, "image/jpeg") }
        response = requests.post(OCR_SERVER, files=files)
        if response.status_code==200:
            res_string = response.json()["string"]
            return res_string
        else:
            return None

def xywh2xyxy(xywh, original_width, original_height):
    """
    Convert [x, y, w, h] in percentage format to xyxy absolute coordinates.

    Parameters:
    xywh (list): List of [x, y, w, h] as percentages, where x and y represent the top-left corner.
    original_width (int): Original width of the image.
    original_height (int): Original height of the image.

    Returns:
    tuple: (x_min, y_min, x_max, y_max) in absolute coordinates.
    """
    x_percent, y_percent, width_percent, height_percent = xywh

    x_min = (x_percent / 100) * original_width
    y_min = (y_percent / 100) * original_height
    x_max = x_min + (width_percent / 100) * original_width
    y_max = y_min + (height_percent / 100) * original_height

    return x_min, y_min, x_max, y_max

def get_img_from_studio(image_path, save_path="./weapon_img"):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    download_url = f"{LABEL_STUDIO_URL}{image_path}"
    headers = {
        'Authorization': f'Token {LABEL_STUDIO_KEY}'
    }
    response = requests.get(download_url, headers=headers)
    if response.status_code == 200:
        filename = os.path.basename(image_path)
        save_path = os.path.join(save_path, filename)
        with open(save_path, 'wb') as f:
            f.write(response.content)
            return save_path
        print(f"Image successfully downloaded as {filename}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")
        return None

In [3]:
with open("./bbox_with_string.json", "r") as loadjson:
    data = json.loads(loadjson.read())
pprint(data[12])

{'annotations': [{'completed_by': 4,
                  'created_at': '2024-11-07T03:16:00.590537Z',
                  'draft_created_at': '2024-11-07T03:14:22.886803Z',
                  'ground_truth': False,
                  'id': 9200,
                  'import_id': None,
                  'last_action': None,
                  'last_created_by': None,
                  'lead_time': 109.496,
                  'parent_annotation': None,
                  'parent_prediction': None,
                  'prediction': {},
                  'project': 12,
                  'result': [{'from_name': 'label',
                              'id': 'npt37v1_XV',
                              'image_rotation': 0,
                              'origin': 'manual',
                              'original_height': 22,
                              'original_width': 85,
                              'to_name': 'image',
                              'type': 'rectanglelabels',
                           

In [4]:
# get all annotations with string pairs
all_label_pairs = []
for anno in data:
    for _a in anno["annotations"][0]["result"]:
        if _a["type"] == "textarea":
            all_label_pairs.append(anno)
pprint(all_label_pairs[0])

{'annotations': [{'completed_by': 1,
                  'created_at': '2024-11-07T10:02:09.227810Z',
                  'draft_created_at': '2024-11-07T10:01:03.922716Z',
                  'ground_truth': False,
                  'id': 10210,
                  'import_id': None,
                  'last_action': None,
                  'last_created_by': None,
                  'lead_time': 289.98199999999997,
                  'parent_annotation': None,
                  'parent_prediction': None,
                  'prediction': {},
                  'project': 12,
                  'result': [{'from_name': 'label',
                              'id': '_d0oE7rshw',
                              'image_rotation': 0,
                              'origin': 'manual',
                              'original_height': 20,
                              'original_width': 133,
                              'to_name': 'image',
                              'type': 'rectanglelabels',
              

In [5]:
char_bbox = []
for _annotation in tqdm(all_label_pairs):
    all_bboxes = []
    label_string = None
    for _anno in _annotation["annotations"][0]["result"]:
        final_pairs = []
        db_fname = _annotation["data"]["image"]
        image_name = get_img_from_studio(db_fname, "/home/hbdesk/labelstudio_convert/bbox_string/")
        original_image = cv2.imread(image_name)
        if len(_annotation["annotations"][0]["result"]) > 0:
            if _anno["type"] == "textarea":
                label_string = _anno["value"]["text"][0]
            if _anno["type"] == "rectanglelabels":
                height = _anno["original_height"]
                width = _anno["original_width"]
                xywh = [_anno["value"]["x"], _anno["value"]["y"], _anno["value"]["width"], _anno["value"]["height"]]
                x1 , y1 , x2, y2  = xywh2xyxy(xywh, width, height)
                all_bboxes.append([x1,y1,x2,y2])
    # print("ALL BOXES: ", all_bboxes)
    print("LABEL: ", label_string)
    if label_string is not None:
        for idx, box in enumerate(all_bboxes):
            try:
                _label_string = label_string.replace(" ", "")
                # print("curchar : ", _label_string)
                curchar = _label_string[idx]
                x1,y1,x2,y2 = [int(x) for x in box]
                crop = original_image[y1:y2, x1:x2]
                h, w ,c = crop.shape
                pad_width = max(0, 32 - w)
                pad_height = max(0, 32 - h)
                top = pad_height // 2
                bottom = pad_height - top
                left = pad_width // 2
                right = pad_width - left
                _pad = np.pad(crop, ((top, bottom), (left, right), (0, 0)), mode='constant', constant_values=0)
                # _resize = cv2.cvtColor(cv2.resize(crop, (112, 112)), cv2.COLOR_BGR2GRAY)
                _pad = cv2.cvtColor(_pad, cv2.COLOR_BGR2GRAY)
                if not os.path.exists(f"bbox_chars/{curchar}"):
                    os.makedirs(f"bbox_chars/{curchar}")
                cv2.imwrite(f"bbox_chars/{curchar}/{uuid.uuid4().hex}.png", _pad)
            except Exception as _:
                continue
    else: 
        continue

              

  0%|                                                                                                       | 1/1021 [00:01<22:28,  1.32s/it]

LABEL:  [RRR] DEMIDDDROR


  0%|▏                                                                                                      | 2/1021 [00:01<15:02,  1.13it/s]

LABEL:  [PERO] HomieG-


  0%|▎                                                                                                      | 3/1021 [00:02<13:54,  1.22it/s]

...

LABEL:  [RAZE] PkChee


100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 1020/1021 [13:11<00:00,  1.22it/s]

LABEL:  [RAZE] Twerktle


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1021/1021 [13:12<00:00,  1.29it/s]

LABEL:  [RAZE] steep_van


In [6]:
print(char_bbox[0])

IndexError: list index out of range